# LLM-as-a-Judge による選好データ合成

Susumu Ota  2025-02-02

本ハンズオンでは、言語モデルを使って選好データ(preference data)を合成する方法を紹介します。

まず、簡単に [Direct Preference Optimization (DPO)](https://arxiv.org/abs/2305.18290) について説明します。次に、[LLM-as-a-Judge](https://arxiv.org/abs/2306.05685) 手法の解説、そして実際に選好データを合成する方法を紹介します。作成した選好データは DPO などの事後学習に利用することができます。

## Direct Preference Optimization (DPO) とは

TODO

<img src="https://github.com/user-attachments/assets/bba0a5d1-b6ba-4c21-aaf7-697169b85db8" width="800">


## LLM-as-a-Judge 手法の概要

TODO


<img src="https://github.com/user-attachments/assets/853e351d-f35b-41f3-be28-29426feede89" width="800">
<br />
<img src="https://github.com/user-attachments/assets/12111a26-50c8-439c-a23a-ae7240c259bc" width="800">
<br />

## 準備

準備については、`Persona-Hub による事後学習データ合成` ハンズオンの内容とほぼ同じですのでそちらの資料を参照してください。

### トークンと API キーの設定

In [1]:
if str(get_ipython()).startswith("<google.colab"):  # if this notebook is running in Google Colab  # type: ignore
    import os
    from google.colab import userdata  # type: ignore

    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    # os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    # os.environ["NVIDIA_NIM_API_KEY"] = userdata.get("NVIDIA_NIM_API_KEY")

### モジュールのインストール

In [2]:
# %pip install litellm
# %pip install datasets
# %pip install pandas

# %pip install vllm

### モジュールのインポート

In [3]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, INFO, StreamHandler, getLogger  # noqa: F401
import pprint


from datasets import load_dataset
from litellm import batch_completion
import pandas as pd


try:
    from vllm import LLM, SamplingParams  # type: ignore
except ImportError:
    print("No vllm module found. You can only use the LiteLLM.")

/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


No vllm module found. You can only use the LiteLLM.


### ログの設定

In [4]:
logging_level = DEBUG
# logging_level = INFO  # uncomment if you want to see less output
logger = getLogger(__name__)
logger.setLevel(logging_level)
handler = StreamHandler()
handler.setLevel(logging_level)
logger.addHandler(handler)

pp = pprint.PrettyPrinter(width=100)

### 言語モデルの設定

`n` というパラメータを追加して、一つのプロンプトに対して複数の結果を生成するよう変更しました。結果出力もリストのリストに変更となります。 `# changed` というコメントがある行が変更された行です。

In [5]:
class LanguageModel(ABC):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None):  # changed
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        self.n = n  # changed
        logger.debug(f"model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}, n: {n}")  # changed

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        pass


class LiteLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None):  # changed
        super().__init__(model, temperature, max_tokens, seed, n)  # changed

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        contents = [
            [choice.message.content for choice in response.choices]  # changed
            for response in batch_completion(
                model=self.model,
                messages=messages_batch,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
                n=self.n,  # changed
            )
        ]
        assert len(contents) == len(messages_batch)
        return contents


class VLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, n=None, dtype="auto", stop=None):  # changed
        super().__init__(model, temperature, max_tokens, seed, n)  # changed
        self.dtype = dtype
        self.stop = stop
        self.vllm = LLM(model, dtype=dtype)  # dtype must be "half" to run on Colab T4
        self.tokenizer = self.vllm.get_tokenizer()

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[list[str]]:  # changed
        sampling_params = SamplingParams(
            temperature=self.temperature, max_tokens=self.max_tokens, seed=self.seed, n=self.n, stop=self.stop  # changed
        )
        prompts = [
            self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            for messages in messages_batch
        ]
        outputs = self.vllm.generate(prompts, sampling_params=sampling_params, use_tqdm=False)
        contents = [
            [oo.text for oo in o.outputs]  # changed
            for o in outputs
        ]
        assert len(contents) == len(messages_batch)
        return contents

### 言語モデルの作成

裁判官の役割をする言語モデルを作成します。温度パラメータ `temperature` が `0.0` であることに注意してください。

Note: 裁判官のタスクは難易度が高いので、高度な言語モデルを使用する必要があります。10B 程度の言語モデルでは難しいかもしれません(タスクを理解できない可能性があります)。

In [6]:
judge_llm = LiteLLMModel("gpt-4o-mini", temperature=0.0, max_tokens=512, seed=0)  # OPENAI_API_KEY
# judge_llm = LiteLLMModel("nvidia_nim/nvidia/nemotron-4-340b-instruct", temperature=0.0, max_tokens=512, seed=None)  # NVIDIA_NIM_API_KEY
# judge_llm = LiteLLMModel("nvidia_nim/nvidia/llama-3.1-nemotron-70b-instruct", temperature=0.0, max_tokens=512, seed=None)  # NVIDIA_NIM_API_KEY
# judge_llm = LiteLLMModel("deepinfra/nvidia/Llama-3.1-Nemotron-70B-Instruct", temperature=0.0, max_tokens=512, seed=0)  # DEEPINFRA_API_KEY
# judge_llm = VLLMModel("hpprc/gemma-2-2b-jpn-it", temperature=0.0, max_tokens=512, seed=0, stop=["<end_of_turn>"], dtype="half")  # for Colab T4
# judge_llm = VLLMModel("marcsun13/gemma-2-9b-it-GPTQ", temperature=0.0, max_tokens=512, seed=0)  # for Colab T4
# judge_llm = VLLMModel("cyberagent/calm3-22b-chat", temperature=0.0, max_tokens=512, seed=0)  # for A100?

judge_llm

model: gpt-4o-mini, temperature: 0.0, max_tokens: 512, seed: 0, n: None


In [7]:
judge_llm([[{"role": "user", "content": "Hello?"}]])

[['Hello! How can I assist you today?']]

## プロンプトの読み込み

プロンプトは事前に用意したものを利用します。

In [8]:
!wget https://raw.githubusercontent.com/susumuota/synthetic-data-hands-on/refs/heads/main/notebooks/basic_math_mt.jsonl

--2025-01-31 06:13:43--  https://raw.githubusercontent.com/susumuota/synthetic-data-hands-on/refs/heads/main/notebooks/basic_math_mt.jsonl
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 12635 (12K) [text/plain]
`basic_math_mt.jsonl.1' に保存中

basic_math_mt.jsonl 100%[===================>]  12.34K  --.-KB/s 時間 0s         

2025-01-31 06:13:43 (25.5 MB/s) - `basic_math_mt.jsonl.1' へ保存完了 [12635/12635]



ここでは Persona-Hub で生成したマルチターンの対話データから最初の質問だけを取り出しプロンプトとします。

In [9]:
def load_questions(input_jsonl: str) -> list[str]:
    dataset = load_dataset("json", data_files=input_jsonl, name="default", split="train", cache_dir="cache")
    batch = pd.DataFrame(dataset).to_dict(orient="records")  # convert column-wise to row-wise
    return [b["messages"][0]["content"] for b in batch]

In [10]:
questions = load_questions("basic_math_mt.jsonl")
questions

Generating train split: 10 examples [00:00, 1628.41 examples/s]


['ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
 'あるプログラミング言語の研究者が、数値計算のアルゴリズムをテストするために、10個の異なる入力データセットを用意しました。彼は各データセットに対して3つの異なるアルゴリズムを試すことにしました。全てのデータセットとアルゴリズムの組み合わせを考えると、彼は合計で何回テストを実施することになりますか？',
 'ある流体力学のエンジニアは、水槽の自由表面の面積を計算しています。水槽の形は長方形で、長さが6メートル、幅が4メートルです。この水槽が満水のとき、水面の面積は何平方メートルですか？',
 'ある年、サーモンが川を遡上するために、1日あたり30キロメートル進みました。サーモンが川を上り始めてから、合計で6日間進みました。サーモンは合計で何キロメートル上ったでしょうか？',
 'ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
 '環境ジャーナリストであるあなたは、オーガニック農法で作物を育てる農家を取材しています。農家は、1エーカーの土地で年間に2000ポンドのオーガニック野菜を収穫します。また、1エーカーの土地に蜂の巣箱を5つ設置すると、収穫量が10％増加します。あなたは、蜂の巣箱を3つ設置した場合の収穫量を計算したいと思っています。蜂の巣箱を3つ設置したときの収穫量は何ポンドになりますか？',
 'ある小学校の先生

## サンプリングで 1個の質問から2個の異なる解答を生成

### 温度パラメータの探索

サンプリングで1個の質問に対して2個の異なる解答を生成します。ただし、解答が同じにならないように温度パラメータを調整します。以下のような方針で探索を行います。

- 1個の質問から2個の解答を生成
- 生成された2個の解答を比較
- 2個の解答が異なっていたらOK
- もし一致していたらNG、温度を少し上げて再度解答を生成

現在学習しようとしている言語モデル(`target_llm`)を初期化します(裁判官言語モデルとは別)。必ず `n=2` としてください。

`target_llm` は現在学習をしようとしているモデルですので、必ずしも高度な言語モデルである必要はありません。

In [11]:
target_llm = LiteLLMModel("gpt-4o-mini", temperature=0.0, max_tokens=512, seed=0, n=2)  # OPENAI_API_KEY

model: gpt-4o-mini, temperature: 0.0, max_tokens: 512, seed: 0, n: 2


下のセルの `target_llm.temperature = 0.7` の部分を低い温度から始めて、全ての解答が異なるまで温度を徐々に上げて最適な `temperature` を探してください。上げすぎるとハルシネーションを起こしたり、文が破綻するので注意してください。

以下のセルを何度か手動で実行してみて、`differences` の要素全て True になることを確認してください。多少 False が出力されても問題ありませんが、False が多い場合は温度を調整してください。

In [12]:
target_llm.temperature = 0.7
target_llm.n = 2

answers = target_llm([[{"role": "user", "content": question}] for question in questions])

differences = [answer[0] != answer[1] for answer in answers]
all(differences), differences

(True, [True, True, True, True, True, True, True, True, True, True])

ここでは `temperature` を `0.7` 程度に設定していますが、最適な `temperature` は言語モデルによって異なります。

### 解答を２個生成

以下のような関数で解答を2個生成します。

In [13]:
def generate_answers(llm: LanguageModel, questions: list[str]) -> list[str]:
    llm.n = 2
    answers = llm([[{"role": "user", "content": question}] for question in questions])
    return [
        {"question": question, "answer_a": answer[0], "answer_b": answer[1]}
        for question, answer in zip(questions, answers)
    ]

実行して確認します。

In [14]:
answers = generate_answers(target_llm, questions)
answers

[{'question': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
  'answer_a': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'answer_b': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。'},
 {'question': 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
  'answer_a': 'クモの化石を見た見学者の数を \\( C \\)、サソリの化石を見た見学者の数を \\( S \\) とします。\n\n問題文から、クモの化石を見た見学者の数 \\( C \\) は20人であり、また、「クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多い」とあるので、次の式が成り立ちます。\n\n\\[\nC = 2S\n\\]\n\nこれに \\( C = 20 \\) を代入すると、\n\n\\[\n20 = 2S\n\\]\n\nこの方程式を \\( S \\) に

2個の解答が異なるかどうかチェックします。

In [15]:
def check_answers(answers: list[dict[str, str]]) -> list[bool]:
    return [a["answer_a"] != a["answer_b"] for a in answers]

In [16]:
results = check_answers(answers)
all(results), results

(True, [True, True, True, True, True, True, True, True, True, True])

もし同じ解答が多い(False の数が多い)場合は、言語モデルの設定で温度パラメータを上げて調整してください。

## LLM-as-a-Judge でどちらの解答が優れているか判定

ここからは、生成された2個の解答を LLM-as-a-Judge 手法でどちらの解答が優れているか判決を下します。

今回は [LLM-as-a-Judge 論文](https://arxiv.org/abs/2306.05685)の [プロンプト](https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl)の `pair-v2` というバージョンを使用して、2つの解答を比較します。

<img src="https://github.com/user-attachments/assets/12111a26-50c8-439c-a23a-ae7240c259bc" width="800">

In [17]:
# copy from https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl

JUDGE_PROMPT = {
    "name": "pair-v2",
    "type": "pairwise",
    "system_prompt": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: \"[[A]]\" if assistant A is better, \"[[B]]\" if assistant B is better, and \"[[C]]\" for a tie.",
    "prompt_template": "[User Question]\n{question}\n\n[The Start of Assistant A's Answer]\n{answer_a}\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\n{answer_b}\n[The End of Assistant B's Answer]",
    "description": "Prompt for general questions",
    "category": "general",
    "output_format": "[[A]]"
}

上記の `system_prompt` と `prompt_template` を使用します。

システムプロンプトの日本語訳は以下の通りです。

> 公平な裁判官として、以下に表示されたユーザーの質問に対して2人のAIアシスタントが提供した回答の質を評価してください。あなたは、ユーザーの指示に従い、ユーザーの質問によりよく答えるアシスタントを選ぶべきです。あなたの評価は、回答の有用性、関連性、正確性、深さ、創造性、詳細レベルなどの要素を考慮する必要があります。2つの回答を比較し、簡単な説明をすることから評価を始めてください。立場が偏らないようにし、回答の提示順があなたの判断に影響しないようにしてください。回答の長さが評価に影響しないようにしてください。特定のアシスタントの名前を好まないこと。できるだけ客観的であること。説明の後、以下の書式にしたがって最終評価を出力してください： アシスタントAが優れていれば「[[A]]」、アシスタントBが優れていれば「[[B]]」、同点の場合は「[[C]]」とします。

判決文を生成する関数を定義します。

In [18]:
def generate_decision_texts(llm: LanguageModel, answers: list[dict[str, str]]) -> list[str]:
    return [texts[0] for texts in llm([[
        {"role": "system", "content": JUDGE_PROMPT["system_prompt"]},
        {"role": "user", "content": JUDGE_PROMPT["prompt_template"].format(**answer)}
    ] for answer in answers])]

実際に判定をしてみます。判決文(`decision_texts`)の末尾に `[[A]]`, `[[B]]`, `[[C]]` のいずれかが含まれているかを確認してください。

Note: このタスクは難易度が高いので、`judge_llm` は高度な言語モデルを使用する必要があります。

In [19]:
decision_texts = generate_decision_texts(judge_llm, answers)
decision_texts

["Both Assistant A and Assistant B provided identical responses to the user's question, calculating the total number of plants collected by the researcher. They both listed the number of plants collected at each location, performed the addition correctly, and arrived at the same conclusion of 54 plants.\n\nIn terms of helpfulness, relevance, accuracy, depth, creativity, and level of detail, both responses are equal. They both clearly outline the steps taken to arrive at the answer and provide the correct total.\n\nSince there is no distinction in the quality of the responses, the evaluation results in a tie.\n\nFinal verdict: [[C]]",
 "Both Assistant A and Assistant B provide clear and accurate solutions to the problem presented by the user. They both correctly identify the relationship between the number of visitors who saw the spider fossils and the scorpion fossils, and they both arrive at the same conclusion that 10 visitors saw the scorpion fossils.\n\n**Comparison:**\n- **Clarity

解答と判決文を表示して中身を確認します。

In [20]:
pp.pprint(list(zip(answers, decision_texts)))

[({'answer_a': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n'
               '\n'
               '地点A: 12本  \n'
               '地点B: 15本  \n'
               '地点C: 8本  \n'
               '地点D: 10本  \n'
               '地点E: 9本  \n'
               '\n'
               'これらを合計すると、\n'
               '\n'
               '12 + 15 + 8 + 10 + 9 = 54\n'
               '\n'
               'したがって、彼が集めた植物の総数は54本です。',
   'answer_b': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n'
               '\n'
               '地点A: 12本  \n'
               '地点B: 15本  \n'
               '地点C: 8本  \n'
               '地点D: 10本  \n'
               '地点E: 9本  \n'
               '\n'
               'これらを合計すると、\n'
               '\n'
               '12 + 15 + 8 + 10 + 9 = 54\n'
               '\n'
               'したがって、彼が集めた植物の総数は54本です。',
   'question': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？'},
  "Both Assis

判決文をパースして、`A`, `B`, `C` のいずれかを返す関数を定義します。明確に決められないケースは `C` とします。

In [21]:
def parse_decision_text(decision_text):
    is_a = is_b = is_c = False
    if "[[A]]" in decision_text:
        is_a = True
    if "[[B]]" in decision_text:
        is_b = True
    if "[[C]]" in decision_text:
        is_c = True
    decision = "C"
    if is_a and is_b:
        # raise ValueError(f"Both A and B are chosen: {decision_text}")
        logger.debug(f"Both A and B are chosen: {decision_text}")
        decision = "C"
    elif is_a:
        decision = "A"
    elif is_b:
        decision = "B"
    elif is_c:
        decision = "C"
    else:
        # raise ValueError(f"Unknown decision: {decision_text}")
        logger.debug(f"Unknown decision: {decision_text}")
        decision = "C"
    # logger.debug(f"decision: {decision}")
    return decision

解答をジャッジして判決を返す関数にまとめます。

In [22]:
def judge_answers(llm: LanguageModel, answers: list[dict[str, str]]) -> list[str]:
    return [parse_decision_text(decision_text) for decision_text in generate_decision_texts(llm, answers)]

実行して、解答に対する判定を確認します。

In [23]:
decisions = judge_answers(judge_llm, answers)
decisions

['C', 'C', 'A', 'A', 'A', 'A', 'B', 'B', 'C', 'B']

ここまでで、1個の質問に対して2個の解答を生成し、それらの解答を LLM-as-a-Judge で判定することができました。

この判定結果から、引き分け `C` 判定を取り除いて `A` と `B` の判決をした解答のみから選好データを生成することになりますが、その前に位置バイアスの影響を取り除く処理を行います。

## 位置バイアスの軽減

位置バイアスとは、言語モデルがある特定の位置を他の位置よりも好む傾向を示すことです。このバイアスは言語モデル特有というわけではなく、人間の意思決定や他の機械学習領域でも見られます([LLM-as-a-Judgeの論文](https://arxiv.org/abs/2306.05685)より)。

例えば、最初に提示された解答が後に提示された解答よりも高い評価を受ける可能性が高い、というバイアスが存在することが知られています。

<img src="https://github.com/user-attachments/assets/3f3d8c4b-0efe-47a7-8d63-1cd8f672e50c" width="600">

<br />

<img src="https://github.com/user-attachments/assets/4b4c4441-8ea6-4e06-9154-a69d49767eed" width="600">

<br />

位置バイアスの影響を減らすための簡単な解決策として、解答の順番をスワップして再度ジャッジして、判決に一貫性がある場合のみ勝利とするという方法があります。スワップした後で結果が一致しない場合は、引き分けとします。

今回はこの方法を使って位置バイアスの影響を軽減します。

まず、解答の順番を入れ替える関数を定義します。

In [24]:
def swap_answers(answers: list[dict[str, str]]) -> list[dict[str, str]]:
    return [{"question": a["question"], "answer_a": a["answer_b"], "answer_b": a["answer_a"]} for a in answers]

スワップしてから判定した結果は A と B の順番が変わっていますので、比較する際は A と B を入れ替えて比較します。判決を入れ替える関数を定義します。

In [25]:
def swap_decisions(decisions: list[str]) -> list[str]:
    return ["A" if d == "B" else "B" if d == "A" else "C" for d in decisions]

実際にスワップして判定してみます。

In [26]:
swapped_decisions = swap_decisions(judge_answers(judge_llm, swap_answers(answers)))
swapped_decisions

['C', 'A', 'B', 'A', 'A', 'C', 'A', 'A', 'C', 'B']

元の判定とスワップした判定を比較して、判決が一致していて、かつ、`C` が含まれていない場合を選好データとして採用します。

In [27]:
def verify_decisions(decisions: list[str], swapped_decisions: list[str]) -> list[list[str]]:
    return [d1 == d2 and d1 != "C" for d1, d2 in zip(decisions, swapped_decisions)]

In [28]:
verifications = verify_decisions(decisions, swapped_decisions)
list(zip(verifications, decisions, swapped_decisions))

[(False, 'C', 'C'),
 (False, 'C', 'A'),
 (False, 'A', 'B'),
 (True, 'A', 'A'),
 (True, 'A', 'A'),
 (False, 'A', 'C'),
 (False, 'B', 'A'),
 (False, 'B', 'A'),
 (False, 'C', 'C'),
 (True, 'B', 'B')]

True が位置を入れ替えても判決がゆるがない(バイアスを受けていない)という結果です、False が位置を入れ替えると判決が覆ってしまう(位置バイアス)場合と引き分けの場合です。

位置をスワップしても判決が変わらなかったのは約半数ということになりました。

ここでの検討事項は、

- 判決が入れ替わった数がどの程度あるか (バイアスあるいはタスクの理解ができていない)
- A (あるいは B)を選ぶ傾向が明確にあるか (バイアスがあるかどうか)
- C を含むケースを選好データから除外するかどうか (例えば `(False, B, C)` のようなケースをBの勝ちとするか、あるいは除外するか)

となります。

今回は、引き分け C を含むケースを全て除外した上で、バイアスを受けなかったデータを選好データとして採用します (`verified` が `True` のデータ)。今までの処理を関数にまとめます。

In [29]:
def generate_preferences(judge_llm: LanguageModel, target_llm: LanguageModel, questions: list[str]) -> list[dict[str, str | bool]]:
    answers = generate_answers(target_llm, questions)
    logger.debug(f"len(answers): {len(answers)}")
    decisions = judge_answers(judge_llm, answers)
    logger.debug(f"decisions: {decisions}")
    swapped_decisions = swap_decisions(judge_answers(judge_llm, swap_answers(answers)))
    logger.debug(f"swapped_decisions: {swapped_decisions}")
    verifications = verify_decisions(decisions, swapped_decisions)
    logger.debug(f"verifications: {verifications}")
    return [
        {
            "prompt": answer["question"],
            "chosen": answer["answer_a"] if decision == "A" else answer["answer_b"],
            "rejected": answer["answer_b"] if decision == "A" else answer["answer_a"],
            "decision": decision,
            "swapped_decision": swapped_decision,
            "verification": verification,
        }
        for answer, decision, swapped_decision, verification in zip(answers, decisions, swapped_decisions, verifications)
    ]

In [30]:
preferences = generate_preferences(judge_llm, target_llm, questions)
preferences

len(answers): 10
decisions: ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'C']
swapped_decisions: ['C', 'A', 'C', 'A', 'A', 'B', 'B', 'A', 'C', 'B']
verifications: [False, False, False, True, True, False, False, False, False, False]


[{'prompt': 'ニュー・ジャージー・パインバレンズの生態系を研究している植物研究者がいます。彼は、特定の植物のサンプルを集めるために、5つの異なる地点を訪れました。各地点で、彼は次のように植物を収集しました：地点Aで12本、地点Bで15本、地点Cで8本、地点Dで10本、地点Eで9本です。彼が集めた植物の総数はいくつですか？',
  'chosen': '植物研究者が集めた植物の総数を求めるために、各地点で収集した植物の本数を足し合わせます。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'rejected': '植物研究者が集めた植物の総数を計算するために、各地点での植物の本数を足します。\n\n地点A: 12本  \n地点B: 15本  \n地点C: 8本  \n地点D: 10本  \n地点E: 9本  \n\nこれらを合計すると、\n\n12 + 15 + 8 + 10 + 9 = 54\n\nしたがって、彼が集めた植物の総数は54本です。',
  'decision': 'C',
  'swapped_decision': 'C',
  'verification': False},
 {'prompt': 'ある博物館で、古代の節足動物の化石が展示されています。展示されている化石は、クモが10体、サソリが15体、エビが20体です。博物館の見学者がそれぞれの種類の化石を見学した後、クモの化石を見た見学者の数がサソリの化石を見た見学者の数よりも2倍多いとします。クモの化石を見た見学者が20人だった場合、サソリの化石を見た見学者は何人ですか？',
  'chosen': 'クモの化石を見た見学者の数が20人で、これがサソリの化石を見た見学者の数よりも2倍多いとします。\n\nサソリの化石を見た見学者の数を \\( x \\) とすると、以下の関係が成り立ちます：\n\n\\[\n20 = 2x\n\\]\n\nこの式を解くと、\n\n\\[\nx = \\frac{20}{2} = 10\n\\]\n\nしたがって、サソリの化石を見た見

jsonl 形式でデータを読み込み、結果を jsonl 形式で保存する関数を作成します。除外したデータは `skip_jsonl` ファイルに保存します。

In [31]:
def run_generate_preferences(judge_llm: LanguageModel, target_llm: LanguageModel, input_jsonl: str = "input.jsonl", output_jsonl: str = "output.jsonl", skip_jsonl: str = "skip.jsonl") -> None:
    questions = load_questions(input_jsonl)
    logger.debug(f"len(questions): {len(questions)}")
    preferences = generate_preferences(judge_llm, target_llm, questions)
    logger.debug(f"len(preferences): {len(preferences)}")
    with open(output_jsonl, "w", encoding="utf-8") as f_output, open(skip_jsonl, "w", encoding="utf-8") as f_skip:
        for preference in preferences:
            f = f_output if preference["verification"] else f_skip
            f.write(json.dumps(preference, ensure_ascii=False) + "\n")
            print(pp.pformat(preference)) if preference["verification"] else None

In [32]:
run_generate_preferences(judge_llm, target_llm, "basic_math_mt.jsonl", "verified_preference.jsonl", "skipped_preference.jsonl")

len(questions): 10
len(answers): 10
decisions: ['C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'B']
swapped_decisions: ['C', 'C', 'C', 'A', 'C', 'B', 'B', 'B', 'C', 'B']
verifications: [False, False, False, True, False, True, True, True, False, True]
len(preferences): 10


{'chosen': '水槽の自由表面の面積は、長方形の面積を計算することで求められます。長方形の面積は、長さと幅を掛け合わせることで計算できます。\n'
           '\n'
           '与えられた情報は以下の通りです：\n'
           '- 長さ = 6 メートル\n'
           '- 幅 = 4 メートル\n'
           '\n'
           '面積 \\( A \\) は次のように計算されます：\n'
           '\n'
           '\\[\n'
           'A = \\text{長さ} \\times \\text{幅} = 6 \\, \\text{m} \\times 4 \\, \\text{m} = 24 \\, '
           '\\text{m}^2\n'
           '\\]\n'
           '\n'
           'したがって、水槽が満水のときの水面の面積は \\( 24 \\, \\text{平方メートル} \\) です。',
 'decision': 'A',
 'prompt': 'ある流体力学のエンジニアは、水槽の自由表面の面積を計算しています。水槽の形は長方形で、長さが6メートル、幅が4メートルです。この水槽が満水のとき、水面の面積は何平方メートルですか？',
 'rejected': '水槽の形が長方形である場合、水面の面積は長さと幅を掛け合わせて計算します。\n'
             '\n'
             '水槽の長さは6メートル、幅は4メートルですので、\n'
             '\n'
             '面積 = 長さ × 幅 = 6メートル × 4メートル = 24平方メートル\n'
             '\n'
             'したがって、水槽が満水のとき、水面の面積は24平方メートルです。',
 'swapped_decision': 'A',
 'verification': True}
{'chosen': 'ロボットの移動速度は1時間で10キロメートルです。もしロボットが30時間移動し続けた場合、移動する距離は以下

位置バイアスを取り除いた選好データが `verified_preference.jsonl` に保存され、除外したデータが `skipped_preference.jsonl` に保存されます。

この `verified_preference.jsonl` を使って、DPO を行い、言語モデルの性能を向上させることができます。

さらに DPO で学習したモデルを使って、再度新たな選好データを合成して学習を行うことで、反復的に性能を向上させる [Iterative RLHF (Iterative DPO)](https://arxiv.org/abs/2405.07863) という手法もあります。


## まとめ

TODO

## 参考文献

TODO